# MFCCS
Obtener los mfccs para 5 clases c/u de 500 audios, para un total de 15.000 audios

Realizar agrupaciones de 10 ventanas y promediar sus coeficientes MFCCS. Nota: el ultimo grupo de ventanas sera en varias ocasiones menor a 10, sin embargo se promediaran.

In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing

In [125]:
#PATHSAMPLES = '../data/training_list_test_temporal.txt'
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 10
SOUNDS =np.array(['yes','no','right','five','nine'])
#SOUNDS =np.array(['off','five','left'])

In [126]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [127]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [128]:
path_word_speakerId.shape

(50, 3)

In [129]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ['data/yes/00f0204f_nohash_0.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_1.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_2.wav', 'yes', '00f0204f'],
       ['data/yes/012c8314_nohash_0.wav', 'yes', '012c8314'],
       ['data/yes/0132a06d_nohash_0.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_1.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_2.wav', 'yes', '0132a06d'],
       ['data/no/012c8314_nohash_0.wav', 'no', '012c8314'],
       ['data/no/0132a06d_nohash_0.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_1.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_2.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_3.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_4.wav', 'no', '0132a06d'],
       ['data/no/013

In [130]:
def build_features(path_word_speakerId, n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)
    matrix_features = np.empty((1,23)) # inicialmente

    #Encode etiqueta del path audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    #Encode
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
    #Recoremos las muestras y sacamos los MFCCS
    
    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        #Los mfccs del audio i-esimo
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        
        n_windows = mfccs.shape[1]
        
        n_groups = int(n_windows/10)
        
        n_windos_end_group = n_windows%10
        
        if(n_windos_end_group !=0):        
            n_groupsT =n_groups + 1
        else:
            n_groupsT = n_groups
        
        #info sobre el audio
        path = sound_info[0]
        speaker_id = sound_info[2]
        word=sound_info[1]      
        
        #Transformación del encoder
        labelPath = lePath.transform([path])
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])                
        
        #Repeticion de las columnas path,word,speaker, multiples instancias        
        
        
        labelPath = np.repeat(labelPath,n_groupsT)
        labelWord =  np.repeat(labelWord,n_groupsT)
        labelSpeaker= np.repeat(labelSpeaker,n_groupsT)        
        
        fila_P_W_S = np.array([labelPath,labelWord,labelSpeaker]).T  
        
        mfccs =mfccs.T
        
        
        first_wind = 0
        last_wind =n_windows-n_windos_end_group
        
        avg_splis = []
        for g in range(n_groups):
            first_wind = g*10
            end_wind = first_wind + 10
            group_1 = mfccs[first_wind:end_wind,:]
            avg_spli_group_1 = np.average(group_1,axis=0)
            avg_splis.append(avg_spli_group_1)
      
        if(n_windos_end_group!=0):
            end_group_1 = n_windows - n_windos_end_group
            group_2 =  mfccs[end_group_1:,:]
            avg_spli_group_2 = np.average(group_2,axis=0)
            avg_splis.append(avg_spli_group_2)
        
        mfccFull =  np.hstack((avg_splis,fila_P_W_S))
        
        
        matrix_features = np.concatenate((matrix_features,mfccFull), axis=0) 
                   
    return matrix_features[1:,:]

In [131]:
matrixFinal = build_features(path_word_speakerId, 20,NSAMPLES, PATHSAMPLES, SOUNDS)
print(matrixFinal.shape)

(540, 23)


In [132]:
np.savetxt('../data/audios_MFCC_average.csv',matrixFinal,delimiter=",")